In [2]:
import pandas as pd

In [3]:
import pickle
import seaborn as sns 
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge 

from sklearn.metrics import root_mean_squared_error

In [5]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747213814820, experiment_id='1', last_update_time=1747213814820, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = (df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']).dt.total_seconds() / 60
    
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [7]:
df_train = read_dataframe('./data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2025-02.parquet')

In [8]:
len(df_train), len(df_val)

(46307, 44218)

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values 
y_val = df_val[target].values 

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

6.098881864144634

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv,lr), f_out)


In [ ]:

with mlflow.start_run():
    mlflow.set_tag("developer", "posi")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2025-01.parquet")
    mlflow.log_param("val-data-path", "./data/green_tripdata_2025-02.parquet")

    alpha = 0.1 
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path = "models/lin_reg.bin", artifact_path="models_pickle")

In [12]:
import xgboost as xgb

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [13]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
"""
# get the best hyperparameters using hyperopt
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params,
            dtrain = train,
            num_boost_round = 1000,
            evals = [(valid, "validation")],
            early_stopping_rounds = 20
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status": STATUS_OK}
"""

In [ ]:
"""
# get the best hyperparameters using hyperopt
search_space = {
    "max_depth": scope.int(hp.quniform("max_depth", 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn = objective,
    space = search_space,
    algo = tpe.suggest,
    max_evals = 50,
    trials=Trials()
)
"""

In [ ]:
"""
# Train the model on the best hyperparameters 
params = {
    'learning_rate':    0.07420190149003122,
    'max_depth':    14,
    'min_child_weight':    3.4147431198015323,
    'objective':    'reg:linear',
    'reg_alpha':    0.03551910752729691,
    'reg_lambda':    0.05407553294541204,
    'seed':    42
}

mlflow.xgboost.autolog(disable = True)

booster = xgb.train(
            params,
            dtrain = train,
            num_boost_round = 1000,
            evals = [(valid, "validation")],
            early_stopping_rounds = 30
        )
"""

2025/05/19 06:42:04 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ada01ae5e84340479d1913f530c8fd84', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:8.69875


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [06:42:06] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:8.33336
[2]	validation-rmse:8.00545
[3]	validation-rmse:7.71228
[4]	validation-rmse:7.45053
[5]	validation-rmse:7.21801
[6]	validation-rmse:7.01188
[7]	validation-rmse:6.83019
[8]	validation-rmse:6.66835
[9]	validation-rmse:6.52627
[10]	validation-rmse:6.40016
[11]	validation-rmse:6.28849
[12]	validation-rmse:6.19246
[13]	validation-rmse:6.10679
[14]	validation-rmse:6.03117
[15]	validation-rmse:5.96440
[16]	validation-rmse:5.90598
[17]	validation-rmse:5.85496
[18]	validation-rmse:5.80946
[19]	validation-rmse:5.76984
[20]	validation-rmse:5.73527
[21]	validation-rmse:5.70366
[22]	validation-rmse:5.67630
[23]	validation-rmse:5.65207
[24]	validation-rmse:5.63083
[25]	validation-rmse:5.61039
[26]	validation-rmse:5.59317
[27]	validation-rmse:5.57750
[28]	validation-rmse:5.56375
[29]	validation-rmse:5.55057
[30]	validation-rmse:5.53871
[31]	validation-rmse:5.52878
[32]	validation-rmse:5.51992
[33]	validation-rmse:5.51158
[34]	validation-rmse:5.50354
[35]	validation-rmse:5.

2025/05/19 06:42:51 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/05/19 06:42:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [06:42:51] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/05/19 06:42:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [16]:
with mlflow.start_run():
    best_params = {
    'learning_rate':    0.07420190149003122,
    'max_depth':    14,
    'min_child_weight':    3.4147431198015323,
    'objective':    'reg:linear',
    'reg_alpha':    0.03551910752729691,
    'reg_lambda':    0.05407553294541204,
    'seed':    42
    }

    # Log the parameters
    mlflow.log_params(best_params)

    booster = xgb.train(
            best_params,
            dtrain = train,
            num_boost_round = 1000,
            evals = [(valid, "validation")],
            early_stopping_rounds = 30
        )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    # Log your DictVectorizer as an artifact 
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path = "models_mlflow")

[0]	validation-rmse:8.69875


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:49:50] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:8.33336
[2]	validation-rmse:8.00545
[3]	validation-rmse:7.71228
[4]	validation-rmse:7.45053
[5]	validation-rmse:7.21801
[6]	validation-rmse:7.01188
[7]	validation-rmse:6.83019
[8]	validation-rmse:6.66835
[9]	validation-rmse:6.52627
[10]	validation-rmse:6.40016
[11]	validation-rmse:6.28849
[12]	validation-rmse:6.19246
[13]	validation-rmse:6.10679
[14]	validation-rmse:6.03117
[15]	validation-rmse:5.96440
[16]	validation-rmse:5.90598
[17]	validation-rmse:5.85496
[18]	validation-rmse:5.80946
[19]	validation-rmse:5.76984
[20]	validation-rmse:5.73527
[21]	validation-rmse:5.70366
[22]	validation-rmse:5.67630
[23]	validation-rmse:5.65207
[24]	validation-rmse:5.63083
[25]	validation-rmse:5.61039
[26]	validation-rmse:5.59317
[27]	validation-rmse:5.57750
[28]	validation-rmse:5.56375
[29]	validation-rmse:5.55057
[30]	validation-rmse:5.53871
[31]	validation-rmse:5.52878
[32]	validation-rmse:5.51992
[33]	validation-rmse:5.51158
[34]	validation-rmse:5.50354
[35]	validation-rmse:5.

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:50:28] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/05/27 15:50:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [18]:
logged_model = 'runs:/219d70c35e1c4ca7af586611063e3cc8/models_mlflow'

"""
# Load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model.predict(pd.DataFrame(valid))
"""

# Load model as an xgboost model
xgboost_model = mlflow.xgboost.load_model(logged_model)
y_pred = xgboost_model.predict(valid)


In [19]:
# The top 10 predictions
y_pred[:10]

array([ 5.2897205, 24.860758 , 26.35573  , 14.33655  ,  9.355858 ,
       14.133669 ,  5.962356 , 22.963251 , 22.055073 , 10.563749 ],
      dtype=float32)